In [44]:
import networkx as nx
import pandas as pd
import numpy as np
import pulp
df = pd.read_csv("dataset/finished_dataset.csv")

# Load the graph from the Pickle file
graph_from_routes = nx.read_gpickle("graph_from_routes_wd.pkl")


In [45]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_date,location_pickup,location_dropoff,pickup_graph_node,dropoff_graph_node
0,0,0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016-03-14,5,22,42435716,42447076
1,1,42,id2129090,1,2016-03-14 14:05:39,2016-03-14 14:28:05,1,-73.975090,40.758766,-73.953201,40.765068,N,1346,2016-03-14,12,11,42445926,42429694
2,2,58,id0256505,1,2016-03-14 15:04:38,2016-03-14 15:16:13,1,-73.994484,40.745087,-73.998993,40.722710,N,695,2016-03-14,30,4,42437890,42458267
3,3,103,id3502065,1,2016-03-15 21:03:44,2016-03-15 21:14:37,1,-73.974014,40.757153,-73.949738,40.777092,N,653,2016-03-15,12,59,42445926,42448254
4,4,374,id3817493,2,2016-03-14 14:57:56,2016-03-14 15:15:26,1,-73.952881,40.766468,-73.978630,40.761921,N,1050,2016-03-14,11,29,42429694,42440012


In [46]:
graph = graph_from_routes.copy()

In [47]:
distances = {}

# Iterate over edges and add distances to the dictionary
for u, v, d in graph.edges(data=True):
    distances[(u, v)] = d.get('weight', None)

In [48]:
for i, node in enumerate(graph.nodes()):
    graph.nodes[node]['id'] = i

In [49]:
def distance_matrix_from_graph(G):
    """
    Create a distance matrix from a networkx graph.
    """
    num_nodes = len(G.nodes)
    distance_matrix = [[0] * num_nodes for _ in range(num_nodes)]
    for u in range(num_nodes):
        for v in range(num_nodes):
            if u != v:
                distance_matrix[u][v] = nx.shortest_path_length(G, source=u, target=v, weight='weight')
    return distance_matrix



In [50]:
class Vehicle():
    def __init__(self, id, pcapacity=4,gcapacity = 10, 
                 discharge_rate = 1.5, 
                 start_depot = 0, 
                 end_depot = 0 ):
        self.id = id
        self.pcapacity_ = pcapacity
        self.gcapacity_ = gcapacity
        self.r_ = discharge_rate
        self.charge = 100
        self.start_depot = start_depot
        self.end_depot = end_depot
        self.requests_ ={}
    def update_requests(self, id, start, dest):
        self.requests_[id] = (start,dest)
    def update_requests(self, request):
        self.requests_[request.id] = (request.start,request.dest)

    def update_charge(self,distance, speed = 50):
        self.charge -=  distance/speed * self.r_


In [51]:
from enum import Enum
class Charger(Enum):
    SLOW = (1,0.6)
    MEDIUM = (1,0.8)
    FAST = (2,0.8)

class Depot():
    def __init__(self, node_id, type = Charger.SLOW, vcapacity = 50, vehicles_now = 25) -> None:
        self.id = node_id
        self.r, self.omega = type.value
        self.vcapacity_ = vcapacity
        self.vehicles_now = vehicles_now


In [52]:
class Request():
    def __init__(self, id, people, start, end) -> None:
        self.id = id
        self.people_ = people
        self.start = start
        self.end = end 


In [53]:
def convert_reqs_to_obj(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    reqs_obj = {}
    for node, item in requests.items():
        amount = item["amount"]
        temp_reqs = []
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
        reqs_obj[node] = temp_reqs
    
    return reqs_obj

In [54]:
def convert_reqs_to_list(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    temp_reqs = []
    for node, item in requests.items():
        amount = item["amount"]
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
    print(len(temp_reqs))
    return temp_reqs

In [55]:
def get_requests(dataframe, required_info = ["amount","internal_ids", "id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    requests = {}
    pick_up_nodes = np.unique(dataframe["pickup_graph_node"])
    
    for node in pick_up_nodes:
        temp_dict = dataframe[dataframe["pickup_graph_node"]==node].to_dict()
        temp_dict["amount"] = len(temp_dict["id"])
        temp_dict["internal_ids"] = list(temp_dict["id"].keys())
        requests[node] = {x:temp_dict[x] for x in required_info}
    reqs_obj = convert_reqs_to_obj(requests)
    reqs_list = convert_reqs_to_list(requests)
    return requests, reqs_obj, reqs_list

In [56]:
def initialize_depots(graph, vehicles_amount = [20]*5,type = [Charger.FAST]*5 ):
    depots = []
    idx = 0
    for node_id, data in graph.nodes(data = True):
        if data["depot"]:
            depots += [Depot(node_id, type = type[idx], vehicles_now = vehicles_amount[idx])]
            idx += 1
    return depots

In [57]:
def initialize_vehicles_naive(depots):
    vehicles = []
    #Calculate the amount of vehicles at the beginning
    n_vehicles = sum(d.vehicles_now for d in depots)
    for i,d in enumerate(depots):
        vehicles+= [Vehicle(id = i ,
                            pcapacity=6, 
                            gcapacity=10, 
                            discharge_rate=1,
                            start_depot= d.id,
                             end_depot= d.id) for i in range(d.vehicles_now*(i), d.vehicles_now * (i+1))]
    return vehicles, n_vehicles

In [58]:
depots = initialize_depots(graph, vehicles_amount = [10]*5, type = [Charger.FAST]*5)
vehicles, n_vehicles = initialize_vehicles_naive(depots)

In [59]:
for d in depots:
    print(d.id)

42447076
6223969260
42448162
42426665
42447330


In [364]:
_, _, requests = get_requests(df[:20])
for r in requests:
    print(r.id, r.start, r.end, r.people_)

20
id1286147 42428598 42440012 1
id3507184 42429330 42442451 2
id3817493 42429694 42440012 1
id0871223 42429694 42445256 1
id3860883 42429694 42445950 1
id3295265 42434098 42447330 1
id2875421 42435716 42447076 1
id3276198 42435716 42430989 1
id3570508 42435716 561042200 2
id2272789 42436914 42442959 5
id0256505 42437890 42458267 1
id3163239 42440966 42445926 1
id3267154 42442514 42448254 1
id2129090 42445926 42429694 1
id3502065 42445926 42448254 1
id1965717 42446994 42446521 1
id0103178 42447076 42445926 1
id0671892 42448162 42458267 1
id3604983 42449948 42442913 1
id2969048 42453104 42446521 2


In [365]:
def make_double_sense(G):
    for u in G.nodes():
        for v in G.nodes():
            # Check if the edge (u, v) does not already exist and if u != v
            if u != v and G.has_edge(u, v):
                # Add the edge (u, v) and (v, u)
                G.add_edge(v, u)
    return G

#graph = make_double_sense(graph)

In [366]:
edges_to_remove = [(u, v) for u, v in graph.edges() if u == v]
graph.remove_edges_from(edges_to_remove)

In [367]:
edges_to_remove

[]

In [368]:
def create_problem(graph,depots, vehicles, requests,distances=distances, name="VehicleRoutingProblem"):
    problem = pulp.LpProblem(name, pulp.LpMinimize)

    #Routing variable
    V = {}
    for a in vehicles:
        for u in graph.nodes():
            for v in graph.nodes():
                if graph.has_edge(u, v):
                    V[a.id,u, v]= pulp.LpVariable(f"V^{a.id}_{u}_{v}", cat='Binary') 
    
    #Request Assigning variable
    x = {}
    for a in vehicles:
        for r in requests:
            x[a.id,r.id]= pulp.LpVariable(f"x_{a.id},{r.id}", cat='Binary') 

    # Define the objective function
    problem += 3000*pulp.lpSum(-x[a.id,r.id] for a in vehicles for r in requests)
    #problem += pulp.lpSum(V[a.id,u, v]*distances[(u,v)] for a in vehicles for v in graph.nodes() for u in graph.nodes() if graph.has_edge(u, v))


    #(4.5)
    for a in vehicles:
            problem += pulp.lpSum(x[a.id,r.id]*r.people_ for r in requests) <= a.pcapacity_

    #(4.9)
    for r in requests:
        problem += pulp.lpSum(x[a.id,r.id] for a in vehicles) == 1

        
    #(4.16)
    for a in vehicles:
        temp_list = list(graph.nodes()).copy()
        temp_list.remove(a.start_depot)
        if a.start_depot != a.end_depot:
            temp_list.remove(a.end_depot)

        for v in temp_list:
            problem += pulp.lpSum(V[a.id,u, v] for u in graph.nodes() if graph.has_edge(u, v)) \
                    - pulp.lpSum(V[a.id,v, w] for w in graph.nodes() if graph.has_edge(v, w)) == 0
    #(4.17), #(4.18)
    #for a in vehicles:
    #    for r in requests:
            #problem += pulp.lpSum(V[a.id,a.start_depot, v] for v in graph.nodes() if graph.has_edge(a.start_depot, v)) >= x[a.id, r.id]
    #        problem += pulp.lpSum(V[a.id, v, a.end_depot] for v in graph.nodes() if graph.has_edge(v, a.end_depot)) == pulp.lpSum(V[a.id,a.start_depot, v] for v in graph.nodes() if graph.has_edge(a.start_depot, v))== x[a.id, r.id]

    
    
    
    #(4.19), #(4.20)
    """       NO ASSIGNMENT 
    for a in vehicles:
        problem += pulp.lpSum(V[a.id,a.start_depot, v] for v in temp_list if graph.has_edge(a.start_depot, v)) ==1
        if a.start_depot != a.end_depot:
            problem += pulp.lpSum(V[a.id,a.end_depot, v] for v in temp_list if graph.has_edge(a.end_depot, v)) ==1
    """
    
    for a in vehicles:
        for r in requests:
            #problem += pulp.lpSum(V[a.id, a.start_depot, v] for v in graph.nodes() if graph.has_edge(a.start_depot, v)) == pulp.lpSum(V[a.id, v, a.end_depot] for v in graph.nodes() if graph.has_edge(v, a.end_depot)) #>= x[a.id, r.id]
            # Constraint: If vehicle a is assigned to request r, then at least one edge from any node v to the start of request r must be selected
            problem += (pulp.lpSum(V[a.id, v, a.end_depot] for v in graph.nodes() if graph.has_edge(v, a.end_depot)) \
                        ==pulp.lpSum(V[a.id, v, r.end] for v in graph.nodes() if graph.has_edge(v, r.end)) \
                            ==pulp.lpSum(V[a.id, v, r.start] for v in graph.nodes() if graph.has_edge(v, r.start)) \
                                == pulp.lpSum(V[a.id, a.start_depot, v] for v in graph.nodes() if graph.has_edge(a.start_depot, v)) ) == x[a.id, r.id]
            #problem += (pulp.lpSum(V[a.id, v, r.end] for v in graph.nodes() if graph.has_edge(v, r.end)) == pulp.lpSum(V[a.id, v, r.start] for v in graph.nodes() if graph.has_edge(v, r.start)))#>= x[a.id, r.id]
        
    return problem, V, x

In [369]:
problem, V, x= create_problem(graph,depots, vehicles, requests)

In [370]:
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/864759225e744431830c9a753ae65cfd-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/864759225e744431830c9a753ae65cfd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4775 COLUMNS
At line 139366 RHS
At line 144137 BOUNDS
At line 173638 ENDATA
Problem MODEL has 4770 rows, 29500 columns and 74480 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -60000 - 4.00 seconds
Cgl0004I processed model has 4770 rows, 29260 columns (29260 integer (29260 of which binary)) and 74240 elements
Cutoff increment increased from 1e-05 to 3000
Cbc0038I Initial state - 31 integers unsatisfied sum - 10.928

1

In [371]:
for a in vehicles:
    for r in requests:
        if pulp.value(x[a.id,r.id]) == 1:
            print(f"Vehicle {a.id} assigned to {r.id}")

Vehicle 0 assigned to id0256505
Vehicle 0 assigned to id3267154
Vehicle 3 assigned to id1286147
Vehicle 3 assigned to id2272789
Vehicle 10 assigned to id3507184
Vehicle 10 assigned to id0103178
Vehicle 12 assigned to id3817493
Vehicle 12 assigned to id0871223
Vehicle 12 assigned to id3860883
Vehicle 14 assigned to id3295265
Vehicle 14 assigned to id3163239
Vehicle 15 assigned to id2129090
Vehicle 15 assigned to id3502065
Vehicle 15 assigned to id0671892
Vehicle 15 assigned to id3604983
Vehicle 16 assigned to id1965717
Vehicle 17 assigned to id2969048
Vehicle 26 assigned to id2875421
Vehicle 26 assigned to id3276198
Vehicle 26 assigned to id3570508


In [372]:
for d in depots:
    print(d.id)

42447076
6223969260
42448162
42426665
42447330


In [373]:
for a in vehicles:
    for u, v in graph.edges():
        if pulp.value(V[a.id,u, v]) == 1:
            print(f"Vehicle {a.id} visits {u}-{v}")
            break

Vehicle 0 visits 42442514-42437305
Vehicle 3 visits 42436914-42428598
Vehicle 10 visits 596776173-42445748
Vehicle 12 visits 42429694-42445950
Vehicle 14 visits 42434098-3099326124
Vehicle 15 visits 42445926-2711029280
Vehicle 16 visits 11252642361-42446994
Vehicle 17 visits 42453104-3892037906
Vehicle 26 visits 42435716-42432818


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/5b2d20c6025846b6b39422408215fb34-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/5b2d20c6025846b6b39422408215fb34-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5285 COLUMNS
At line 177616 RHS
At line 182897 BOUNDS
At line 212898 ENDATA
Problem MODEL has 5280 rows, 30000 columns and 83660 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 20206.6 - 0.36 seconds
Cgl0004I processed model has 5280 rows, 30000 columns (30000 integer (30000 of which binary)) and 83660 elements
Cbc0038I Initial state - 20 integers unsatisfied sum - 3.54286
Cbc0038I Pass   1: (1.13 seconds) suminf.  

In [340]:
for u, v in graph.edges():
        if pulp.value(V[11,u, v]) == 1:
            print(f"Vehicle {11} visits {u}-{v}")